In [1]:
from __future__ import print_function

from github_settings import (ry_username, ry_password,
                             username, password,
                             token, 
                             GITENBERG_TRAVIS_TOKEN,
                             RDHYEE_TRAVIS_TOKEN)


from itertools import (islice, izip)
import requests


from second_folio import (apply_to_repos)

from gitenberg_utils import (GitenbergJob,
                             GitenbergTravisJob,
                             ForkBuildRepo,
                             BuildRepo,
                             BuildRepo2,
                             MetadataWrite, 
                             repo_md)


import pandas as pd
from pandas import (DataFrame, Series)

# Nostromo--A-Tale-of-the-Seaboard_2021

[AsciiDoc version by james-bostock · Pull Request #1 · GITenberg/Nostromo--A-Tale-of-the-Seaboard_2021](https://github.com/GITenberg/Nostromo--A-Tale-of-the-Seaboard_2021/pull/1)

In [ ]:
repo_name = 'Nostromo--A-Tale-of-the-Seaboard_2021'

rs = MetadataWrite(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis')

rs.run()

In [ ]:
bj = BuildRepo(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis')

In [ ]:
bj.run()

# status of second folio

In [2]:
from second_folio import (all_repos)

all_repos[:5]

['Adventures-of-Huckleberry-Finn_76',
 'Don-Quixote_996',
 'Dubliners_2814',
 'Jane-Eyre_1260',
 'Moby-Dick--Or-The-Whale_2701']

In [3]:
len(all_repos)

152

In [4]:
repo_name = all_repos[0]

In [5]:
repos = all_repos[:]

def status_for_repo(repo_name):
    rs = GitenbergJob(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis')
    return rs.status()

results_iter = apply_to_repos(status_for_repo, repos=repos)

In [6]:
results = []

for (i,result) in enumerate(results_iter):
    results.append(result)
    print ("\r{}: {}".format(i, result['repo_name']), end="")

151: On-Liberty_34901

In [7]:
df = DataFrame(results)

In [8]:
df.head()

,.travis.deploy.api_key.txt,.travis.yml,asciidocs,book.asciidoc,cover.jpg,ebooks_in_latest_github_release,ebooks_in_release_count,metadata.yaml,repo_name,repo_owner,version
0,True,True,[book.asciidoc],True,True,"[(epub, Adventures-of-Huckleberry-Finn.epub), ...",3,True,Adventures-of-Huckleberry-Finn_76,GITenberg,0.2.0
1,True,True,[book.asciidoc],True,True,"[(epub, Don-Quixote.epub), (mobi, Don-Quixote....",3,True,Don-Quixote_996,GITenberg,0.2.0
2,True,True,[book.asciidoc],True,True,"[(epub, Dubliners.epub), (mobi, Dubliners.mobi...",3,True,Dubliners_2814,GITenberg,0.2.0
3,True,True,[book.asciidoc],True,True,"[(epub, Jane-Eyre-An-Autobiography.epub), (mob...",3,True,Jane-Eyre_1260,GITenberg,0.2.0
4,True,True,[book.asciidoc],True,True,"[(epub, Moby-Dick-Or-The-Whale.epub), (mobi, M...",3,True,Moby-Dick--Or-The-Whale_2701,GITenberg,0.2.0


In [ ]:
import json
print (json.dumps(results))

In [ ]:
df.columns

In [13]:
def my_filter(r):
    #return (r['metadata.yaml'] and r['book.asciidoc']) and (r['ebooks_in_release_count'] == 0) and (not r['.travis.yml'])
    #return (r['ebooks_in_release_count'] == 0) and (r['book.asciidoc']) 
    return (r['ebooks_in_release_count'] == 0) and r['.travis.yml']

In [14]:
len(df[df.apply(my_filter, axis=1)])

1

In [15]:
repos = list(df[df.apply(my_filter, axis=1)]['repo_name'])
repos

['Ulysses_4300']

# Repo name mismatch problem

In [ ]:
import yaml

class RepoNameFixer(BuildRepo):
    def run (self):
        # check whether repo_name in metadata.yaml and .travis.yml match the repo name
        md = self.metadata().metadata
        if md.get('_repo') != self.repo_name:
            # fix metadata.yaml
            md['_repo'] = self.repo_name
            result = self.create_or_update_file(
                 path = "metadata.yaml", 
                 message = "fix repo name in metadata.yaml",
                 content = yaml.safe_dump(md,default_flow_style=False,
                      allow_unicode=True),
                 ci_skip = True)
            
        # do rest of run
        super(RepoNameFixer, self).run()


In [ ]:
repo_name = 'Tess-of-the-d-Urbervilles--a-Pure-Woman_110'

rnf = RepoNameFixer(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis')

In [ ]:
rnf.run()

In [ ]:
for repo in repos[1:]:
    rnf = RepoNameFixer(username=username, password=password, repo_name=repo,
              repo_owner='GITenberg', 
              update_travis_commit_msg='fix repo name',
              tag_commit_message='update travis')
    print (repo, rnf.run())

In [ ]:
'a' == 'a1' == 'a'

In [ ]:
rnf.run()

In [ ]:
import yaml

In [ ]:
import yaml
y = yaml.safe_load( 
   rnf.gh_repo.contents(".travis.yml").decoded
)
y['deploy']['on']['repo']

In [ ]:
repos = ['White-Fang_910',
 'The-Fat-and-the-Thin_5744',
 'The-Communist-Manifesto_61',
 'The-Legend-of-Sleepy-Hollow_41',
 'Pollyanna_1450',
 'Washington-Square_2870',
 'The-Kreutzer-Sonata-and-Other-Stories_689',
 'The-Wind-in-the-Willows_289',
 'The-Moonstone_155',
 'Siddhartha_2500',
 'The-Importance-of-Being-Earnest--A-Trivial-Comedy-for-Serious-People_844',
 'The-Federalist-Papers_1404',
 'The-Man-in-the-Iron-Mask_2759',
 'Uncle-Tom-s-Cabin_203',
 'Jungle-Tales-of-Tarzan_106',
 'The-Beautiful-and-Damned_9830',
 'The-House-of-the-Seven-Gables_77',
 'A-Christmas-Carol_46',
 'Twelve-Years-a-Slave--13-Narrative-of-Solomon-Northup-a-Citizen-of-New-York-Kidnapped-in-Wash__45631',
 'The-Lady-with-the-Dog-and-Other-Stories_13415',
 'A-Doll-s-House---a-play_2542',
 'Emma_158',
 'Northanger-Abbey_121',
 'Thus-Spake-Zarathustra--A-Book-for-All-and-None_1998',
 'The-Age-of-Innocence_541',
 'Oliver-Twist_730',
 'North-and-South_4276']

In [ ]:
repos_to_process =  repos[15:]

def build_repo(repo_name):
    rs = BuildRepo(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis',
              travis_token = GITENBERG_TRAVIS_TOKEN)
    return rs.run()

results_iter = apply_to_repos(build_repo, repos=repos_to_process)

In [ ]:
results = []

for (i,result) in enumerate(results_iter):
    results.append(result)
    print ("\r{}: {}".format(i, result[0]), end="")

In [ ]:
results

In [ ]:
repo_name = repos_to_process[0]
rs = BuildRepo(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis',
              travis_token = GITENBERG_TRAVIS_TOKEN)
rs.run()

# to move to library: MetadataWrite

In [ ]:
import yaml
import gitenberg

"""
https://github.com/rdhyee/gitberg
had to download https://www.gutenberg.org/cache/epub/feeds/rdf-files.tar.bz2
and configure /Users/raymondyee/Library/Application Support/gitberg/config.yaml (Mac)
to pick up Gutenberg metadata 
"""

def repo_md(repo_id):

    b = gitenberg.Book(repo_id)
    b.parse_book_metadata()

    return b.meta.metadata 

class MetadataWrite(GitenbergJob):
    
    def run(self):
        try:
            import gitenberg

            repo_id = self.repo_name.split("_")[-1]
            md = repo_md(repo_id)
            # add version
            md["_version"] = "0.0.1"

            # add keywords 
            subjects = md.get("subjects")
            if 'GITenberg' not in subjects:
                subjects.append('GITenberg')
            md["subjects"] = subjects

            result = self.create_or_update_file(
                             path = "metadata.yaml", 
                             message = "write metadata.yaml",
                             content = yaml.safe_dump(md,default_flow_style=False,
                                  allow_unicode=True),
                             ci_skip = True)
            return (self.repo_name, result)
        except Exception as e:
            return (self.repo_name, e)

In [ ]:
repo_md('1079')

In [ ]:
repos_to_process =  ['The-Life-and-Opinions-of-Tristram-Shandy-Gentleman_1079']

def write_metadata(repo_name):
    rs = MetadataWrite(username=username, password=password, repo_name=repo_name,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis')
    return rs.run()

results_iter = apply_to_repos(write_metadata, repos=repos_to_process)

In [ ]:
results = []

for (i,result) in enumerate(results_iter):
    results.append(result)
    print ("\r{}: {}".format(i, result[0]), end="")

In [ ]:
repos_to_process

In [ ]:
repos[0]

In [ ]:
write_metadata(repos[0])

In [ ]:
yaml.safe_dump(y,default_flow_style=False,
                    allow_unicode=True)

In [ ]:
# https://gist.githubusercontent.com/rdhyee/9665aa23084a45269cd3c941f702602f/raw/9332e96fa5d3c86137d4a5c49c2ba7b484a4f7c1/repos_status.json

# gitenberg for book metadata

# Archival

In [ ]:
for repo in repos2_to_build[:]:
    try:
        bj = BuildRepo2(username=username, password=password, repo_name=repo,
              repo_owner='GITenberg', 
              update_travis_commit_msg='update travis',
              tag_commit_message='update travis',
              travis_token=GITENBERG_TRAVIS_TOKEN)
    
        bj.run()
    except Exception as e:
        print (repo, e)

In [ ]:
from gitenberg import metadata
import os

def local_yaml_file(id_):
    
    fname = "/Users/raymondyee/C/src/gitenberg-dev/giten_site/metadata/{}.yaml".format(id_)
    if os.path.exists(fname):
        md = metadata.pandata.Pandata(fname)
        return md
    else:
        return None


In [ ]:
for repo in all_repos[:]:
    id_ = repo.split("_")[-1]
    yaml_file = local_yaml_file(id_)
    if yaml_file is None:
        print (repo, yaml_file)

# changes to make in metadata file to initialize it

* _version 0.0.1

```
covers:							      
- attribution: Ed Gaither - Modern Electrographic, 2015	      
  cover_type: original					      
  image_path: cover.jpg					      
  rights: Attribution-NonCommercial 4.0 International (CC BY- 
  rights_url: https://creativecommons.org/licenses/by-nc/4.0/ 
```

```
publication_date 
publisher: Recovering the Classics	
rights: CC BY-NC
rights_url http://creativecommons.org/licenses/by-nc/4.0/
```


* add GITenberg subject

In [ ]:
gj.travis_encrypt(RDHYEE_DON_QUIXOTE_TOKEN)

In [ ]:
u = gj.gh.user()
u.email

In [ ]:
gj.fork_and_build_gitenberg_repo()

In [ ]:
gj.create_or_update_file(path='JUNK.md', message=b'updated junk.md', content=u'hello'.encode('utf-8'))

In [ ]:
print(gj.update_travis_template(write_changes=False, 
                                encrypted_key=gj.travis_encrypt(gj.repo_token()))[0])

## next step

* make sure travis_repo is active
* check on existence of metadata.yaml in the the new github repo
* write .travis.deploy.api_key.txt to the github repo
* compute new .travis.yml and write to github repo
* tag repo 

In [ ]:
from travispy import TravisPy


travis = TravisPy.github_auth(RDHYEE_TRAVIS_TOKEN)

t_user = travis.user()
t_user.login

In [ ]:
travis_repo = travis.repo('rdhyee/Don-Quixote_996')
travis_repo.active

travis_repo.enable()

In [ ]:
travis_encrypt(token_to_encrypt=token.token.encode('utf-8'),
               repo_slug="rdhyee/Don-Quixote_996")

# create test parameters for travispy

https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_settings.example.json

In [16]:
import json
from travispy import TravisPy

def create_travispy_test_settings(github_token, repo_slug):
    settings = {}
    
    travis = TravisPy.github_auth(github_token)
    
    settings['github_token'] = github_token
    settings['repo_slug'] = repo_slug
    
    # account
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_authenticated.py#L31
    
    accounts = travis.accounts()
    account = travis.account(accounts[0].id)
    
    settings['account'] = {
        'count': len(accounts),
        'id': account.id,
        'name': account.name,
        'login': account.login,
        'type': account.type,
        'repos_count': account.repos_count,
        'subscribed': hasattr(account, 'subscribed')
    }
    
    # hook
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_authenticated.py#L73
    
    hooks = travis.hooks()
    hook = hooks[0]

    settings['hook'] = {
        'count': len(hooks),
        'name': hook.name,
        'description': hook.description,
        'owner_name': hook.owner_name,
        'active': hook.active,
        'private': hook.private,
        'admin': hook.admin
    }
    
    # user
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_authenticated.py#L110
    
    user = travis.user()

    settings['user'] = {
        'login': user['login'],
        'name': user['name']
    }
    
    # branch
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L19
    branches = travis.branches(slug=repo_slug)
    branch = travis.branch('master', repo_slug)
    
    settings['branch'] = {
        'count': len(branches),
        'id': branch.id,
        'repository_id': branch.repository_id,
        'pull_request': branch.pull_request,
        'config': branch.config,
        'number': branch.number
    }
    
    # build
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L66

    builds = travis.builds(slug=repo_slug)
    build = travis.build(builds[0].id)
    build_id = builds[0].id
    
    settings['build'] = {
        'count': len(builds),
        'id': build.id,
        'repository_id': build.repository_id,
        'number': build.number,
        'pull_request': build.pull_request,
        'pull_request_title': build.pull_request_title,
        'pull_request_number': build.pull_request_number,
        'config': build.config
    }
    
    # commit
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L115
    
    commit = build.commit
    
    settings['commit'] = {
        'count': len(builds),
        'id': commit.id,
        'sha': commit.sha,
        'branch': commit.branch,
        'message': commit.message,
        'committed_at': commit.committed_at,
        'author_name': commit.author_name,
        'author_email': commit.author_email,
        'commiter_name': commit.committer_name, # sic
        'commiter_email': commit.committer_email, # sic
        'compare_url': commit.compare_url,
        'pull_request_number': None if not hasattr(commit, 'pull_request_number') else commit.pull_request_number
    }


    # jobs
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L140
    
 
    jobs = travis.jobs(ids=build.job_ids)
    job = travis.job(build.job_ids[0])
    
    settings['job'] = {
        'count': len(jobs),
        'build_id': job.build_id,
        'repository_id': job.repository_id,
        'number': job.number,
        'config': job.config,
        'queue': job.queue,
        'allow_failure': job.allow_failure,
        'annotation_ids': job.annotation_ids
        
    }
    
    # repo
    # https://github.com/menegazzo/travispy/blob/v0.3.4/travispy/_tests/test_not_authenticated.py#L252
    # let's add fake stuff for now
    
    settings['repo'] =  {
        "public_count": 25,
        "member_count": 5,
        "owner_count": 7,
        "github_language": "Python",
        "id": 2598876,
        "description": "TravisPy test project for Python 2.7",
        "active": True
  }

    
    return json.dumps(settings, indent=2)

In [17]:
print(create_travispy_test_settings(RDHYEE_TRAVIS_TOKEN, 'rdhyee/hello-travis-ci'))

/Users/raymondyee/C/src/travispy/travispy/entities/_entity.py:196: UserWarning: Unknown Commit attribute branch_is_default
  .format(entity.__class__.__name__, key))


{
  "account": {
    "count": 9, 
    "name": "Raymond Yee", 
    "subscribed": false, 
    "type": "user", 
    "login": "rdhyee", 
    "repos_count": 127, 
    "id": 134212
  }, 
  "github_token": "ee9ac39e8e95d998217a4f8477ac6df1a2921b6d", 
  "repo": {
    "description": "TravisPy test project for Python 2.7", 
    "member_count": 5, 
    "public_count": 25, 
    "github_language": "Python", 
    "owner_count": 7, 
    "active": true, 
    "id": 2598876
  }, 
  "hook": {
    "count": 168, 
    "owner_name": "EbookFoundation", 
    "name": "fef-questionnaire", 
    "admin": true, 
    "private": false, 
    "active": null, 
    "description": ""
  }, 
  "job": {
    "count": 1, 
    "build_id": 93062803, 
    "queue": "builds.gce", 
    "repository_id": 4634765, 
    "config": {
      "dist": "trusty", 
      "language": "ruby", 
      "script": [
        "python -V", 
        "which python", 
        "python -c \"import sys; print(sys.path)\"", 
        "/usr/bin/python test.py"
   